In [6]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from time import mktime
import time
import re
from io import StringIO
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


In [9]:
def get_data(ticker=str,begin=str,end=str):
    
    import time
    begin_time = time.mktime(datetime.strptime(begin, "%Y%m%d").timetuple())
    end_time = time.mktime(datetime.strptime(end, "%Y%m%d").timetuple())
    url = 'https://finance.yahoo.com/quote/{0}/history?period1={1}&period2={2}&interval=1mo&filter=history&frequency=1mo'.format(ticker,int(begin_time),int(end_time))
#     url = 'https://finance.yahoo.com/quote/{0}/history?period1=1551398400&period2=1582934400&interval=1d&filter=history&frequency=1d'.format(ticker)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    table_tag = soup.find('table', class_='W(100%) M(0)')
    table_tag = BeautifulSoup(str(table_tag), 'html.parser')
    rows = table_tag.find_all('tr', attrs={'class':'BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)'})

    df = pd.DataFrame()
    for row in rows:
        time = row.find_all('td', attrs={'class':'Py(10px) Ta(start) Pend(10px)'})
        cells = row.find_all('td', attrs={'class':'Py(10px) Pstart(10px)'})
    # print(cells)
        try: 
            d = {
            'time' : time[0].text,
            'Open' : cells[0].text,
            'High': cells[1].text,
            'Low' : cells[2].text,
            'Close' : cells[3].text,
            'Adj Close**' : cells[4].text,
            'Ticker' : ticker
            }
        except:
            continue
        df = df.append(d, ignore_index=True) 
    df['time'] = pd.to_datetime(df['time']).dt.strftime('%Y%m%d')
    cols=[i for i in df.columns if i not in ["time",'Ticker']]
    for col in cols:
        df[col]=pd.to_numeric(df[col])
    df = df.sort_values('time', ascending = True).reset_index(drop=True)
    return df  
    

In [ ]:
def download_data(tickers,begin=str,end=str):
    df = pd.DataFrame()
    for tick in tickers:
        df2 = get_data(tick,begin,end)
        df = pd.concat([df,df2])
    return df

In [7]:
tickers = ['ALFEN.AS','AF.PA','BESI.AS','SLR.MC']

In [8]:
df = download_data(tickers,'20191001','20200630')

In [11]:
get_data('ALFEN.AS','20191001','20200701')

,Adj Close**,Close,High,Low,Open,Ticker,time
0,14.74,14.74,15.42,12.80,13.10,ALFEN.AS,20191001
1,13.90,13.90,15.20,12.92,14.82,ALFEN.AS,20191101
2,16.44,16.44,16.84,13.68,14.00,ALFEN.AS,20191201
3,24.25,24.25,27.40,15.92,16.44,ALFEN.AS,20200101
4,26.65,26.65,30.30,23.95,24.25,ALFEN.AS,20200201
5,21.05,21.05,29.15,16.20,27.50,ALFEN.AS,20200301
6,26.30,26.30,26.85,20.50,20.65,ALFEN.AS,20200401
7,27.85,27.85,28.80,23.75,25.50,ALFEN.AS,20200501
8,35.90,35.90,36.90,27.65,28.35,ALFEN.AS,20200601
9,37.00,37.00,43.75,34.75,36.05,ALFEN.AS,20200701
